In [12]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../data/fare_amount_scr_dst_t.csv')

In [5]:
df

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
0,0,00:00:00,2.0,2.0,10.500000,10.50,2
1,0,00:00:00,3.0,69.0,24.500000,24.50,1
2,0,00:00:00,4.0,4.0,8.475000,4.50,20
3,0,00:00:00,4.0,7.0,22.000000,20.75,6
4,0,00:00:00,4.0,13.0,13.625000,13.50,8
...,...,...,...,...,...,...,...
755359,95,23:45:00,263.0,259.0,31.000000,31.00,1
755360,95,23:45:00,263.0,260.0,17.840909,17.50,22
755361,95,23:45:00,263.0,261.0,26.300000,25.50,5
755362,95,23:45:00,263.0,262.0,4.942857,4.50,70


In [133]:
# time*pickup*dropoff 96*263*263
mc_mtx = np.zeros((max(df['pickup_datetime_index'])+1, 263, 263))
for i in range(len(df)):
    t = df['pickup_datetime_index'][i]
    pickup = int(df['pickup_taxizone_id'][i]) -1
    dropoff = int(df['dropoff_taxizone_id'][i]) -1
    mc_mtx[t, pickup, dropoff] = df['count'][i]   

In [154]:
mc_mtx_prob = np.zeros((max(df['pickup_datetime_index'])+1, 263, 263))
for i in range(mc_mtx.shape[0]):
    mc_mtx_prob[i] = mc_mtx[i]/np.sum(mc_mtx[i], axis = 1).reshape(263, 1)
mc_mtx_prob = np.nan_to_num(mc_mtx_prob)

C:\Users\stell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [135]:
#check
df.loc[((df['pickup_taxizone_id'] == 217) & (df['pickup_datetime_index'] ==33)),:]

,pickup_datetime_index,pickup_datetime_interval,pickup_taxizone_id,dropoff_taxizone_id,mean,median,count
266288,33,08:15:00,217.0,61.0,9.0,9.0,1
266289,33,08:15:00,217.0,80.0,4.0,4.0,1
266290,33,08:15:00,217.0,97.0,11.0,11.0,1
266291,33,08:15:00,217.0,107.0,13.0,13.0,1


In [155]:
mc_mtx_prob[33,216,:]

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.

In [137]:
from random import choices
population = np.arange(1, 264)
weights = mc_mtx[33,216,:]
num = choices(population, weights)
print(num[0])

107


In [ ]:
# saving reshaped array to file. 
mc_mtx_prob_reshaped = mc_mtx_prob.reshape(mc_mtx_prob.shape[0], -1) 
np.savetxt("../data/mc_mtx.txt", mc_mtx_prob_reshaped)